### The Deutsch-Jozsa algorithm

Implement the $U_f$ gate for a general black box (or oracle) function $f: \{0,1\}^n \rightarrow \{0,1\}$. We don't want to simulate the auxiliary qubit explicitly, so the action of the $U_f$ gate is an added phase $(-1)^{f(x)}$ on each basis state x. Let's be super clear on what we mean by this: The computational basis consists of $2^n$ states: $\{|00..0\rangle, |00..01\rangle.., |11..1\rangle  \}$. The bitstring $x$ that enters into the black box function is the binary array of length $n$ that describes the state of each qubit for a given basis state. We can also enumerate  the basis states with the integer numbers $x$ running from $0$ to $2^n-1$. Now the bitstring $x$ is simply the list of binary digits of the integer number $x$. This identification of the "index" of the basis state with the corresponding "state" of the qubits will also be crucial for the other quantum algorithms like the quantum Fourier transform.

Deutsch–Jozsa: If their can send “quantum enquiry” one query is enough.

How the algorithm work?

1- Initialize two registers
   - n-qubit input in state |0⟩^n
   - 1-qubit output in state |1⟩
2- Apply Hadamard gates to all qubit → put them into superposition
3- Apply the oracle Uf (black-box that compute f(x))
4- Apply Hadamard gates again to input qubit
5- Complete the Algorithm
6- Measured the qubit

In [1]:
from quantum_algorithms import I, X, P0, P1, H, Z, Y, S, T, deutsch_jozsa, rotation_gate, CNOT, controlled_gate, run_dj,  U_N_qubits

In [2]:
import numpy as np

def apply_hadamard_all(n):
    """Apply Hadamard to all n qubits"""
    H = hadamard(1)
    for _ in range(n-1):
        H = np.kron(H, hadamard(1))
    return H

def oracle_matrix(f, n):
    """
    Construct the oracle Uf for Deutsch-Jozsa.
    f: a function f: {0,1}^n -> {0,1}
    n: number of qubits
    """
    N = 2**n
    Uf = np.zeros((N, N))
    for x in range(N):
        y = x ^ (f(x) << 0)  # XOR with f(x) to flip target if needed
        Uf[x, y] = 1
    return Uf

def deutsch_jozsa(f, n, num_samples=1000):
    """
    Deutsch-Jozsa algorithm simulation for n qubits
    f: oracle function
    n: number of qubits
    num_samples: number of measurement samples
    """
    N = 2**n

    # Step 1: Initialize |0>^n
    state = np.zeros(N)
    state[0] = 1

    # Step 2: Apply Hadamard to all qubits
    Hn = apply_hadamard_all(n)
    state = Hn @ state

    # Step 3: Apply Oracle Uf
    Uf = oracle_matrix(f, n)
    state = Uf @ state

    # Step 4: Apply Hadamard to all qubits again
    state = Hn @ state

    # Step 5: Compute probabilities and sample
    probs = np.abs(state)**2
    outcomes = np.arange(N)
    samples = np.random.choice(outcomes, size=num_samples, p=probs)
    return samples, probs

# -----------------------------
# Example oracles
# -----------------------------
def constant_zero(x):
    return 0

def constant_one(x):
    return 1

def balanced_example(x):
    # Example: returns 1 if number of 1s in binary representation is odd
    return bin(x).count("1") % 2

# -----------------------------
# Run the algorithm
# -----------------------------
n_qubits = 3
num_samples = 10000

# Constant oracle
samples_const, probs_const = deutsch_jozsa(constant_zero, n_qubits, num_samples)
print("Constant oracle samples distribution:")
unique, counts = np.unique(samples_const, return_counts=True)
print(dict(zip(unique, counts)))

# Balanced oracle
samples_bal, probs_bal = deutsch_jozsa(balanced_example, n_qubits, num_samples)
print("\nBalanced oracle samples distribution:")
unique, counts = np.unique(samples_bal, return_counts=True)
print(dict(zip(unique, counts)))


Constant oracle samples distribution:
{0: 10000}

Balanced oracle samples distribution:
{0: 10000}
